Several functions are in the train.py
TODO: MOVE THEM TO UTILZE

In [1]:
import glob
from itertools import chain
import os
import argparse
import random
import zipfile
import gc
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
from linformer import Linformer
import torchvision.transforms.functional as TF
from torch.autograd import Variable
import torch.nn.functional as F
import io
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
#from vit_pytorch.efficient import ViT
#from vit_pytorch.efficient import ViT
from vit_pytorch.vit import ViT
from vit_pytorch.deepvit import DeepViT


import wandb

import sys
# import custom utilize

from util.utilize import *
import util.utilize as ut


In [2]:
# lr: 1e-3 3e-3 1e-4 3e-4
# split 0 1 2 run name: backbone lr loss split
lrs, splits = [1e-3, 3e-3, 1e-4, 3e-4], [0, 1, 2]
for lr in lrs:
    for split in splits:
        run_name = f"vit_lr{lr}_split{split}"
        print(run_name)
        wandb.init(project='vit', name=run_name)
        args = {
            'device': torch.device("cuda:0"),
            # 'model': get_model_octa_resume(outsize=5, path='/code/covid_ckpts/octa_split1/1527_val_acc0.694444477558136.pt', dropout=0.15), # 写一个函数获取不同的model和预训练model，这样方便些
            # 'model': get_model_conv(pretrain_out=4,outsize=5, path='/code/covid_ckpts/oct4class_biglr/val_acc0.9759836196899414.pt'),
            'model': get_model_oct_withpretrain(pretrain_out=4,outsize=5, path='/code/covid_ckpts/oct4class_biglr/val_acc0.9759836196899414.pt', dropout=0.15),
            'save_path': f'/code/covid_ckpts/octa_split{split}/', 
            'criterion': mixloss(bce_weight=0.999),
            'epochs': 200,
            'lr': lr,
            'batch_size': 300,
            'datasets': get_dataUNI(split_idx=split),
            'is_echo': False,
            'optimizer': optim.Adam,
            'scheduler': optim.lr_scheduler.CosineAnnealingLR,
            'train_loader': None,
            'eval_loader': None,
            'shuffle': True,
            'is_MIX': True, # 是否使用mixloss input
            # 'wandb': ['visual-intelligence-laboratory','delete_vit3d'],
            'wandb': ['2065136374',run_name]
        }
        ut.device = args['device']
        # do not change the excution 
        args['optimizer'] = args['optimizer'](args['model'].parameters(), lr=args['lr'], weight_decay=1e-3)
        args['scheduler'] = args['scheduler'](args['optimizer'], T_max= args['epochs'], eta_min=8e-6)
        args['train_loader'], args['eval_loader'] = DataLoader(args['datasets'][0], args['batch_size'], args['shuffle']), DataLoader(args['datasets'][1], args['batch_size'], args['shuffle'])
        torch.autograd.set_detect_anomaly(True)
        train_epoch(**args)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


vit_lr0.001_split0


wandb: Currently logged in as: 2065136374. Use `wandb login --relogin` to force relogin


sefl= torch.Size([50, 1024]) torch.Size([1, 1024]) Dropout(p=0.15, inplace=False)
Train dataset length: 700, Val dataset length: 19


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


...

TPR: 0.21052631578947367, TNR: 0.543859649122807


/code/chen/COVID-Net/util/utilize.py:54: RuntimeWarning: invalid value encountered in true_divide
  sensitivity = np.where(tp+fn == 0, 0, tp/(tp+fn))
/code/chen/COVID-Net/util/utilize.py:55: RuntimeWarning: invalid value encountered in true_divide
  specificity = np.where(tn+fp == 0, 0, tn/(tn+fp))


  0%|          | 0/3 [00:00<?, ?it/s]

/opt/conda/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


KeyboardInterrupt: 

In [1]:
#
args = {
    'device': torch.device("cuda:2"),
    # 'model': get_model_octa_resume(outsize=5, path='/code/covid_ckpts/octa_split1/1527_val_acc0.694444477558136.pt', dropout=0.15), # 写一个函数获取不同的model和预训练model，这样方便些
    # 'model': get_model_conv(pretrain_out=4,outsize=5, path='/code/covid_ckpts/oct4class_biglr/val_acc0.9759836196899414.pt'),
    'model': get_model_oct_withpretrain(pretrain_out=4,outsize=5, path='/code/covid_ckpts/oct4class_biglr/val_acc0.9759836196899414.pt', dropout=0.15),
    'save_path': '/code/covid_ckpts/octa_split1/', 
    'criterion': mixloss(bce_weight=1),
    'epochs': 2000,
    'lr': 1e-4,
    'batch_size': 300,
    'datasets': get_dataUNI(),
    'is_echo': False,
    'optimizer': optim.Adam,
    'scheduler': optim.lr_scheduler.CosineAnnealingLR,
    'train_loader': None,
    'eval_loader': None,
    'shuffle': True,
    'is_MIX': True, # 是否使用mixloss input
    # 'wandb': ['visual-intelligence-laboratory','delete_vit3d'],
    'wandb': ['2065136374','delete']
}
ut.device = args['device']
# do not change the excution 
args['optimizer'] = args['optimizer'](args['model'].parameters(), lr=args['lr'], weight_decay=1e-3)
args['scheduler'] = args['scheduler'](args['optimizer'], T_max= args['batch_size'], eta_min=8e-6)
args['train_loader'], args['eval_loader'] = DataLoader(args['datasets'][0], args['batch_size'], args['shuffle']), DataLoader(args['datasets'][1], args['batch_size'], args['shuffle'])
torch.autograd.set_detect_anomaly(True)
train_epoch(**args)

NameError: name 'torch' is not defined